# Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, sys
from pathlib import Path
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break
    
from data_wrangling import specific_utils, process_monkey_information, general_utils
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features
from visualization.matplotlib_tools import plot_behaviors_utils
from neural_data_analysis.neural_analysis_tools.get_neural_data import neural_data_processing
from neural_data_analysis.neural_analysis_tools.visualize_neural_data import plot_neural_data, plot_modeling_result
from neural_data_analysis.neural_analysis_tools.model_neural_data import transform_vars, neural_data_modeling, drop_high_corr_vars, drop_high_vif_vars
from neural_data_analysis.topic_based_neural_analysis.neural_vs_behavioral import prep_monkey_data, prep_target_data, neural_vs_behavioral_class
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural import planning_and_neural_class, pn_utils, pn_helper_class, pn_aligned_by_seg, pn_aligned_by_event
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class, cca_utils, cca_cv_utils
from neural_data_analysis.neural_analysis_tools.cca_methods.cca_plotting import cca_plotting, cca_plot_lag_vs_no_lag, cca_plot_cv
from machine_learning.ml_methods import regression_utils, regz_regression_utils, ml_methods_class, classification_utils, ml_plotting_utils, ml_methods_utils
from planning_analysis.show_planning import nxt_ff_utils, show_planning_utils
from neural_data_analysis.neural_analysis_tools.gpfa_methods import elephant_utils, fit_gpfa_utils, plot_gpfa_utils, gpfa_helper_class
from neural_data_analysis.neural_analysis_tools.align_trials import align_trial_utils
from data_wrangling import combine_info_utils, base_processing_class

import sys
import math
import gc
import subprocess
from pathlib import Path
from importlib import reload

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from scipy import linalg, interpolate
from scipy.signal import fftconvolve
from scipy.io import loadmat
from scipy import sparse
import torch
from numpy import pi
import warnings

# Machine Learning imports
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.multivariate.cancorr import CanCorr

# Neuroscience specific imports
import neo
import rcca
import quantities as pq

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
print("done")

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
done
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Iterate over session

In [5]:
sessions_df_for_one_monkey

,monkey_name,data_name,finished
35,monkey_Schro,data_0316,False
37,monkey_Schro,data_0321,False
46,monkey_Schro,data_0322,False
38,monkey_Schro,data_0326,False
36,monkey_Schro,data_0327,False
39,monkey_Schro,data_0328,False
34,monkey_Schro,data_0329,False
31,monkey_Schro,data_0402,False
33,monkey_Schro,data_0403,False
32,monkey_Schro,data_0404,False


In [8]:
raw_data_dir_name = 'all_monkey_data/raw_monkey_data'
monkey_name = 'monkey_Schro'
sessions_df_for_one_monkey = combine_info_utils.make_sessions_df_for_one_monkey(
        raw_data_dir_name, monkey_name)

# with warnings.catch_warnings():
#     warnings.simplefilter("ignore")
for index, row in sessions_df_for_one_monkey.iterrows():
    if row['finished'] is True:
        continue
    raw_data_folder_path = os.path.join(
        raw_data_dir_name, row['monkey_name'], row['data_name'])
    print('='*100)
    print(raw_data_folder_path)
    pn = pn_aligned_by_event.PlanningAndNeuralEventAligned(raw_data_folder_path=raw_data_folder_path)
    time_resolved_regression.streamline_getting_time_resolved_cv_scores(pn)
    sessions_df_for_one_monkey.loc[index, 'finished'] = True



all_monkey_data/raw_monkey_data/monkey_Schro/data_0316
Retrieved monkey_information
The number of points that were removed due to delta_position exceeding the ceiling is 0
Note: ff_caught_T_sorted is replaced with ff_caught_T_new
Retrieved ff_dataframe from all_monkey_data/processed_data/monkey_Schro/data_0316/ff_dataframe.h5
Made new binned_spikes_df and saved to all_monkey_data/processed_neural_data/monkey_Schro/data_0316/binned_spikes_df_0p1.csv
Retrieved monkey_information
The number of points that were removed due to delta_position exceeding the ceiling is 0
Note: ff_caught_T_sorted is replaced with ff_caught_T_new
Removed 0 rows out of 138 rows where cur_ff was not visible bbas or nxt_ff was not visible both bbas and bsans
shared_stops_near_ff_df has 138 rows
Retrieving shared_stops_near_ff_df succeeded
[Note] When calling modify_position_of_ff_with_big_angle_for_finding_null_arc, Even with `remove_i_o_modify_rows_with_big_ff_angles=False`, some rows may still be removed (e.g., n

Timepoints: 100%|██████████| 10/10 [00:08<00:00,  1.18it/s]


Saved time_resolved_cv_scores to all_monkey_data/planning_and_neural/monkey_Schro/data_0316/event_aligned/time_resolved_cv_scores/precomputed_gpfa/scores_b0p1_t2_cur_first_p0p25_po0p75_d7_cv5.csv
all_monkey_data/raw_monkey_data/monkey_Schro/data_0321
Loaded binned_spikes_df from all_monkey_data/processed_neural_data/monkey_Schro/data_0321/binned_spikes_df_0p1.csv
Loaded planning_data_by_point from all_monkey_data/planning_and_neural/monkey_Schro/data_0321/planning_data_by_point/opt_arc_stop_closest/test/cur_vis_0_1_window_-50cm_0cm
Loaded planning_data_by_point from all_monkey_data/planning_and_neural/monkey_Schro/data_0321/planning_data_by_point/opt_arc_stop_closest/control/cur_vis_0_1_window_-50cm_0cm
Overlap: []
Percentage overlap relative to df1: 0.00%
Percentage overlap relative to df2: 0.00%
Average percentage overlap: 0.00%
Loaded behav_data_by_bin from all_monkey_data/target_decoder/monkey_Schro/data_0321/behav_data_by_point.csv
Retrieved monkey_information
The number of points

Timepoints: 100%|██████████| 10/10 [00:08<00:00,  1.19it/s]


Saved time_resolved_cv_scores to all_monkey_data/planning_and_neural/monkey_Schro/data_0322/event_aligned/time_resolved_cv_scores/precomputed_gpfa/scores_b0p1_t2_cur_first_p0p25_po0p75_d7_cv5.csv
all_monkey_data/raw_monkey_data/monkey_Schro/data_0326
Retrieved monkey_information
The number of points that were removed due to delta_position exceeding the ceiling is 0
Note: ff_caught_T_sorted is replaced with ff_caught_T_new
Retrieved ff_dataframe from all_monkey_data/processed_data/monkey_Schro/data_0326/ff_dataframe.h5
Made new binned_spikes_df and saved to all_monkey_data/processed_neural_data/monkey_Schro/data_0326/binned_spikes_df_0p1.csv
Retrieved monkey_information
The number of points that were removed due to delta_position exceeding the ceiling is 0
Note: ff_caught_T_sorted is replaced with ff_caught_T_new
Removed 0 rows out of 173 rows where cur_ff was not visible bbas or nxt_ff was not visible both bbas and bsans
shared_stops_near_ff_df has 173 rows
Retrieving shared_stops_near

...

Timepoints: 100%|██████████| 10/10 [00:08<00:00,  1.13it/s]


Saved time_resolved_cv_scores to all_monkey_data/planning_and_neural/monkey_Schro/data_0329/event_aligned/time_resolved_cv_scores/precomputed_gpfa/scores_b0p1_t2_cur_first_p0p25_po0p75_d7_cv5.csv
all_monkey_data/raw_monkey_data/monkey_Schro/data_0402
Retrieved monkey_information
The number of points that were removed due to delta_position exceeding the ceiling is 0
Note: ff_caught_T_sorted is replaced with ff_caught_T_new
Retrieved ff_dataframe from all_monkey_data/processed_data/monkey_Schro/data_0402/ff_dataframe.h5
Made new binned_spikes_df and saved to all_monkey_data/processed_neural_data/monkey_Schro/data_0402/binned_spikes_df_0p1.csv
Retrieved monkey_information
The number of points that were removed due to delta_position exceeding the ceiling is 0
Note: ff_caught_T_sorted is replaced with ff_caught_T_new
Removed 0 rows out of 127 rows where cur_ff was not visible bbas or nxt_ff was not visible both bbas and bsans
shared_stops_near_ff_df has 127 rows
Retrieving shared_stops_near

Timepoints: 100%|██████████| 10/10 [00:09<00:00,  1.00it/s]


Saved time_resolved_cv_scores to all_monkey_data/planning_and_neural/monkey_Schro/data_0402/event_aligned/time_resolved_cv_scores/precomputed_gpfa/scores_b0p1_t2_cur_first_p0p25_po0p75_d7_cv5.csv
all_monkey_data/raw_monkey_data/monkey_Schro/data_0403
Retrieved monkey_information
The number of points that were removed due to delta_position exceeding the ceiling is 0
Note: ff_caught_T_sorted is replaced with ff_caught_T_new
Retrieved ff_dataframe from all_monkey_data/processed_data/monkey_Schro/data_0403/ff_dataframe.h5
Made new binned_spikes_df and saved to all_monkey_data/processed_neural_data/monkey_Schro/data_0403/binned_spikes_df_0p1.csv
Retrieved monkey_information
The number of points that were removed due to delta_position exceeding the ceiling is 0
Note: ff_caught_T_sorted is replaced with ff_caught_T_new
Removed 0 rows out of 306 rows where cur_ff was not visible bbas or nxt_ff was not visible both bbas and bsans
shared_stops_near_ff_df has 306 rows
Retrieving shared_stops_near

ValueError: Observation covariance matrix is rank deficient.
Possible causes: repeated units, not enough observations.